In [57]:
import time
import re
import pandas as pd
import numpy as np
from datetime import datetime

In [28]:
from bs4 import BeautifulSoup

# Load the html content from the txt file
file_path = r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\txt files\manual_webscraper_html.txt"

with open (file_path, 'r', encoding = 'utf-8') as file:
    file_content = file.read()

soup = BeautifulSoup(file_content, 'html.parser')


In [29]:
print(soup.prettify())

<html class="notranslate" lang="en" translate="no">
 <head>
  <link href="https://fonts.googleapis.com" rel="preconnect"/>
  <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
  <link as="script" fetchpriority="low" href="/_next/static/chunks/webpack-37dc116c646d133e.js" rel="preload"/>
  <script async="" defer="" src="https://matomo.stratz.com/matomo.js" type="text/javascript">
  </script>
  <script async="" src="/_next/static/chunks/fd9d1056-e4ed69b157b2b858.js">
  </script>
  <script async="" src="/_next/static/chunks/3853-276c7084e681b78c.js">
  </script>
  <script async="" src="/_next/static/chunks/main-app-d25a7f1eba336e98.js">
  </script>
  <link as="script" href="https://polyfill.io/v3/polyfill.min.js?features=Intl.RelativeTimeFormat,Intl.RelativeTimeFormat.~locale.en,ResizeObserver" rel="preload"/>
  <link as="script" href="/dotaconstants?languageEnum=ENGLISH" rel="preload"/>
  <meta content="notranslate" name="google"/>
  <link href="/favicon.ico" rel="s

In [30]:
# Load previous data from csv file
df = pd.read_csv(r"C:\Users\henry\OneDrive - Louisiana State University\Desktop\Dota2Predictor\dota.csv")

In [7]:
# Get MatchID from title

title_tag = soup.find('title').text

matchid = int(title_tag.split(' ')[1])


In [32]:
print(matchid)

7255673780


In [9]:
# Determine which row to add match information

row_index = df.loc[df['MatchID'] == matchid].index         # Take the index of the row with the correct matchid
print(row_index)

Index([9], dtype='int64')


In [10]:
my_hero = df.loc[row_index,'Hero']
my_hero = my_hero.iloc[0].lower().replace(' ','_')

In [77]:
print(my_hero)

venomancer


In [12]:
all_hero_tags = soup.find_all('img', class_ = 'sc-9ad1ed81-0 gWWkux')

radiant_team_tags = all_hero_tags[:5]
dire_team_tags = all_hero_tags[5:10]

In [13]:
radiant_team = []
dire_team = []

#Extract hero name from the image tag and append to enemey_heroes 
for i in radiant_team_tags:
     hero = i['src'].split('/')[6].replace('_horz.png','')
     radiant_team.append(hero)    

for i in dire_team_tags:
     hero = i['src'].split('/')[6].replace('_horz.png','')
     dire_team.append(hero)                           

print(radiant_team)
print(dire_team)


['lone_druid', 'queenofpain', 'elder_titan', 'omniknight', 'venomancer']
['slark', 'windrunner', 'earthshaker', 'pudge', 'spirit_breaker']


In [14]:
if str(my_hero) in radiant_team:
    team = 'Radiant'
    df.loc[row_index, 'Team'] = team
    enemy_heroes = dire_team
    ally_heroes = radiant_team
else:
    team = 'Dire'
    df.loc[row_index, 'Team'] = team
    enemy_heroes = radiant_team
    ally_heroes = dire_team
    

In [15]:
print(ally_heroes)

['lone_druid', 'queenofpain', 'elder_titan', 'omniknight', 'venomancer']


In [16]:
position_number = 1
for enemy_hero, ally_hero in zip(enemy_heroes,ally_heroes) :
    df.loc[row_index,'Enemy_pos' + str(position_number)] = enemy_hero
    if position_number != 5:
        df.loc[row_index,'Ally_pos' + str(position_number)] = ally_hero
    position_number += 1


In [18]:
# Find my user_name tag
user_tag = soup.find('span', text = 'MambaMama')
print(user_tag)

<span class="hitagi__sc-6oal1n-0 eOJEeW"><span class="hitagi__sc-41hgfb-2 fdOBEx"><span class="hitagi__sc-41hgfb-1 hHdoEl"><a href="/players/122262109">MambaMama</a></span></span></span>


C:\Users\henry\AppData\Local\Temp\ipykernel_12112\497887998.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  user_tag = soup.find('span', text = 'MambaMama')


In [19]:
# Extract networth from tag after my user_name then convert 'k' to multiply by 1000
networth = user_tag.find_next('div', class_ = 'hitagi__sc-1ah81hi-0 jhafUR').text.replace('k', '')
networth = float(networth) * 1000
print(networth)

17200.0


In [76]:
# Extract Date and add to row
date_time = soup.find_all('span', class_ = "hitagi__sc-t22b45-1 gvSdww")[10].find('span').text

date_time = datetime.strptime(date_time, '%b %d, %Y, %I:%M %p')

df.loc[row_index, 'Date'] = date_time

NameError: name 'df' is not defined

In [20]:
# Add networth to row
df.loc[row_index,'Networth'] = networth

In [21]:
df.iloc[0]

MatchID               7325788277
Date                  2023-09-08
Hero                      Jakiro
Match Duration             42:19
Result                Lost Match
Kills                          6
Deaths                         8
Assists                       16
Networth                 15500.0
Team                        Dire
Enemy_pos1           drow_ranger
Enemy_pos2                 razor
Enemy_pos3               slardar
Enemy_pos4                treant
Enemy_pos5               warlock
Ally_pos1                 muerta
Ally_pos2             lone_druid
Ally_pos3         spirit_breaker
Ally_pos4               silencer
Name: 0, dtype: object

In [22]:
csv_dota2 = 'dota.csv'
df.to_csv(csv_dota2, index=False)